In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
%cd gdrive/My Drive/UncertaintyModelling/

/content/gdrive/My Drive/UncertaintyModelling


In [ ]:
!pip install pgmpy

In [4]:
import pandas as pd
import numpy as np
import pickle
import pgmpy
import sklearn
from sklearn.metrics import f1_score

In [5]:
with open("models/bayesian_18countries_learnt_bn_treesearch.pickle", "rb") as handle:
  model = pickle.load(handle)

In [6]:
train_df = pd.read_csv("data/train_18_countries.csv")
print(train_df.shape)
train_df.head()

(495, 307)


,deaths_per_mil_cat_india,facial_covering_new_zealand,cancel_public_events_argentina,debt_relief_india,income_support_singapore,school_closures_argentina,vaccine_policy_china,restriction_internal_movement_indonesia,workplace_closures_finland,internation_travel_control_finland,...,cases_per_mil_cat_finland,cases_per_mil_cat_hong_kong,cases_per_mil_cat_indonesia,cases_per_mil_cat_india,cases_per_mil_cat_israel,cases_per_mil_cat_malaysia,cases_per_mil_cat_new_zealand,cases_per_mil_cat_singapore,cases_per_mil_cat_united_states,cases_per_mil_cat_south_africa
0,0,3,2,1.0,2,1,5,2,1,2.0,...,1,0,1,1,5,3,0,1,6,2
1,0,2,2,0.0,2,3,4,2,2,3.0,...,1,0,0,1,4,1,0,1,4,1
2,0,3,1,1.0,2,0,5,2,1,2.0,...,4,0,1,1,6,4,0,3,6,3
3,0,3,2,1.0,2,1,5,2,1,2.0,...,1,0,1,1,5,3,0,1,5,2
4,0,2,2,0.0,2,1,4,2,2,3.0,...,1,0,0,0,4,1,0,1,4,1


In [7]:
test_df = pd.read_csv("data/test_18_countries.csv")
print(test_df.shape)
test_df.head()

(165, 307)


,deaths_per_mil_cat_india,facial_covering_new_zealand,cancel_public_events_argentina,debt_relief_india,income_support_singapore,school_closures_argentina,vaccine_policy_china,restriction_internal_movement_indonesia,workplace_closures_finland,internation_travel_control_finland,...,cases_per_mil_cat_finland,cases_per_mil_cat_hong_kong,cases_per_mil_cat_indonesia,cases_per_mil_cat_india,cases_per_mil_cat_israel,cases_per_mil_cat_malaysia,cases_per_mil_cat_new_zealand,cases_per_mil_cat_singapore,cases_per_mil_cat_united_states,cases_per_mil_cat_south_africa
0,0,3,1,1.0,2,0,5,2,1,2.0,...,4,0,1,1,6,4,0,3,6,3
1,0,3,2,1.0,2,1,5,2,1,2.0,...,1,0,1,1,6,3,0,1,6,2
2,0,2,2,2.0,2,3,0,2,1,3.0,...,0,0,0,0,1,0,0,1,1,1
3,0,3,2,1.0,2,1,5,2,1,2.0,...,1,0,1,1,5,3,0,1,5,2
4,0,0,2,2.0,2,3,0,1,1,3.0,...,0,0,0,0,0,0,0,0,1,0


In [8]:
test_df = test_df.astype(int)

# Inference

In [16]:
from pgmpy.inference import VariableElimination

ve = VariableElimination(model)
print(
    ve.query(
        ["internation_travel_control_singapore"], evidence={"internation_travel_control_hong_kong": 0}
    )
)

/usr/local/lib/python3.7/dist-packages/pgmpy/factors/discrete/DiscreteFactor.py:537: UserWarning: Found unknown state name. Trying to switch to using all state names as state numbers
  UserWarning,


  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

+-------------------------------------------+---------------------------------------------+
| internation_travel_control_singapore      |   phi(internation_travel_control_singapore) |
+===========================================+=============================================+
| internation_travel_control_singapore(2.0) |                                      0.2452 |
+-------------------------------------------+---------------------------------------------+
| internation_travel_control_singapore(3.0) |                                      0.7548 |
+-------------------------------------------+---------------------------------------------+


In [15]:
from pgmpy.inference import VariableElimination

ve = VariableElimination(model)
print(
    ve.query(
        ["internation_travel_control_singapore"], evidence={"internation_travel_control_hong_kong": 1}
    )
)

/usr/local/lib/python3.7/dist-packages/pgmpy/factors/discrete/DiscreteFactor.py:537: UserWarning: Found unknown state name. Trying to switch to using all state names as state numbers
  UserWarning,


  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

+-------------------------------------------+---------------------------------------------+
| internation_travel_control_singapore      |   phi(internation_travel_control_singapore) |
+===========================================+=============================================+
| internation_travel_control_singapore(2.0) |                                      0.1447 |
+-------------------------------------------+---------------------------------------------+
| internation_travel_control_singapore(3.0) |                                      0.8553 |
+-------------------------------------------+---------------------------------------------+


In [20]:
from pgmpy.inference import VariableElimination

ve = VariableElimination(model)
print(
    ve.query(
        ["vaccine_policy_malaysia"], evidence={"vaccine_policy_singapore": 0}
    )
)

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

+----------------------------+--------------------------------+
| vaccine_policy_malaysia    |   phi(vaccine_policy_malaysia) |
+============================+================================+
| vaccine_policy_malaysia(0) |                         0.9609 |
+----------------------------+--------------------------------+
| vaccine_policy_malaysia(1) |                         0.0037 |
+----------------------------+--------------------------------+
| vaccine_policy_malaysia(2) |                         0.0242 |
+----------------------------+--------------------------------+
| vaccine_policy_malaysia(3) |                         0.0104 |
+----------------------------+--------------------------------+
| vaccine_policy_malaysia(4) |                         0.0007 |
+----------------------------+--------------------------------+
| vaccine_policy_malaysia(5) |                         0.0000 |
+----------------------------+--------------------------------+


In [21]:
from pgmpy.inference import VariableElimination

ve = VariableElimination(model)
print(
    ve.query(
        ["vaccine_policy_malaysia"], evidence={"vaccine_policy_singapore": 5}
    )
)

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

+----------------------------+--------------------------------+
| vaccine_policy_malaysia    |   phi(vaccine_policy_malaysia) |
+============================+================================+
| vaccine_policy_malaysia(0) |                         0.0023 |
+----------------------------+--------------------------------+
| vaccine_policy_malaysia(1) |                         0.0030 |
+----------------------------+--------------------------------+
| vaccine_policy_malaysia(2) |                         0.0138 |
+----------------------------+--------------------------------+
| vaccine_policy_malaysia(3) |                         0.0285 |
+----------------------------+--------------------------------+
| vaccine_policy_malaysia(4) |                         0.3953 |
+----------------------------+--------------------------------+
| vaccine_policy_malaysia(5) |                         0.5572 |
+----------------------------+--------------------------------+


**Split by features**

In [9]:
countries = [
    "singapore",
    "china",
    "malaysia",
    "indonesia",
    "hong_kong",
    "australia",
    "new_zealand",
    "united_states",
    "canada",
    "argentina",
    "brazil",
    "south_africa",
    "egypt",
    "germany",
    "finland",
    "switzerland",
    "israel",
    "india"
]

In [10]:
policies = [
            "school_closures",
            "workplace_closures",
            "cancel_public_events",
            "restrict_public_gathering",
            "closure_public_transport",
            "shn_requirement",
            "restriction_internal_movement",
            "internation_travel_control",
            "pi_campaign",
            "testing_policy",
            "contact_tracing",
            "facial_covering",
            "vaccine_policy",
            "income_support",
            "debt_relief"
]

In [11]:
dfs_policy = {}

In [12]:
import regex as re
for policy in policies:
  pattern = "^policy"
  my_regex = r"^" + re.escape(policy)
  dfs_policy[policy] = test_df.filter(regex=(my_regex))

In [13]:
from pgmpy.inference import VariableElimination
ve = VariableElimination(model)

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
fig, ax = plt.subplots(1,1, figsize=(80,80))
nx.draw(model, with_labels=True, ax=ax)
plt.tight_layout()

In [15]:
cdf = dfs_policy['internation_travel_control']

In [16]:
from itertools import permutations

In [17]:
cc = list(permutations(cdf.columns,2))
for c in cc:
  print(c[0], ' ', c[1])
  new_df = pd.concat([cdf[c[0]], cdf[c[1]]], axis=1, keys=[c[0], c[1]])
  print(type(c[0]))
  break
new_df.head()

internation_travel_control_finland   internation_travel_control_canada
<class 'str'>


,internation_travel_control_finland,internation_travel_control_canada
0,2,4
1,2,4
2,3,4
3,2,4
4,3,4


In [18]:
pred_list = {}
for c in cc:
  preds = []

  new_df = pd.concat([cdf[c[0]], cdf[c[1]]], axis=1, keys=[c[0], c[1]])

  for idx, row in new_df.drop([c[1]], axis = 1).iterrows():
    evidence = row.to_dict()
    res = ve.query(
          [c[1]], evidence=evidence, show_progress=False
      )
    ans_dict = {}
    for key, val in zip(res.state_names[res.variables[0]], res.values):
      ans_dict[key] = val
    max_val = max(ans_dict, key=ans_dict.get)
    preds.append(max_val)
  pred_list[c[0]+'-'+c[1]] = preds

In [19]:
predictions_list = {}
for c in cc:
  predictions = []
  for t in pred_list[c[0]+'-'+c[1]]:
    predictions.append(t)
  preds_df = pd.DataFrame(predictions, columns = [c[1]])
  predictions_list[c[0]+'-'+c[1]] = preds_df

In [20]:
scores = {}
for c in cc:
  preds_df = predictions_list[c[0]+'-'+c[1]]
  actual_df = test_df[c[1]]
  score = f1_score(actual_df, preds_df, average='micro')
  scores[c[0]+'-'+c[1]] = score

In [21]:
scores

{'internation_travel_control_argentina-internation_travel_control_australia': 0.9454545454545454,
 'internation_travel_control_argentina-internation_travel_control_brazil': 0.7272727272727273,
 'internation_travel_control_argentina-internation_travel_control_canada': 1.0,
 'internation_travel_control_argentina-internation_travel_control_china': 0.915151515151515,
 'internation_travel_control_argentina-internation_travel_control_egypt': 0.8121212121212121,
 'internation_travel_control_argentina-internation_travel_control_finland': 0.8545454545454545,
 'internation_travel_control_argentina-internation_travel_control_germany': 0.8424242424242424,
 'internation_travel_control_argentina-internation_travel_control_hong_kong': 0.7515151515151516,
 'internation_travel_control_argentina-internation_travel_control_india': 0.6606060606060606,
 'internation_travel_control_argentina-internation_travel_control_indonesia': 0.7333333333333333,
 'internation_travel_control_argentina-internation_travel_

In [22]:
nodes_and_children = [(node, model.get_children(node)) for node in model.nodes]
node_to_children_map = {
    node: children
    for node, children in sorted(
        nodes_and_children, key=lambda el: len(el[1]), reverse=True
    )
}

In [29]:
for idx, (node, children) in enumerate(node_to_children_map.items()):
    if node == "date":
        continue

    print(f"Node - {node}")
    print(f"Number of children - {len(children)}")
    policy = node[: node.rindex("_")]
    print(
        f"Number of related policies in other countries - {[child for child in children if policy in child]}"
    )
    linked_countries = {
        child[child.rindex("_") + 1 :] for child in children if child != "date"
    }
    print(f"Number of linked countries - {len(linked_countries)} ({linked_countries})")
    print()

    if idx == 50:
        break

Node - workplace_closures_germany
Number of children - 68
Number of related policies in other countries - []
Number of linked countries - 18 ({'indonesia', 'switzerland', 'singapore', 'australia', 'israel', 'kong', 'finland', 'germany', 'brazil', 'malaysia', 'states', 'argentina', 'canada', 'africa', 'india', 'china', 'zealand', 'egypt'})

Node - cases_per_mil_cat_argentina
Number of children - 24
Number of related policies in other countries - ['cases_per_mil_cat_canada', 'cases_per_mil_cat_south_africa']
Number of linked countries - 12 ({'indonesia', 'switzerland', 'finland', 'kong', 'germany', 'brazil', 'states', 'canada', 'africa', 'india', 'china', 'egypt'})

Node - cases_per_mil_cat_united_states
Number of children - 23
Number of related policies in other countries - []
Number of linked countries - 11 ({'switzerland', 'australia', 'israel', 'singapore', 'kong', 'germany', 'brazil', 'malaysia', 'states', 'canada', 'africa'})

Node - cases_per_mil_cat_brazil
Number of children - 17

In [7]:
[edge for edge in model.edges if "new_zealand" in edge[0]]

[('school_closures_new_zealand', 'workplace_closures_new_zealand'),
 ('school_closures_new_zealand', 'restriction_internal_movement_new_zealand'),
 ('school_closures_new_zealand', 'closure_public_transport_new_zealand'),
 ('school_closures_new_zealand', 'shn_requirement_new_zealand'),
 ('workplace_closures_new_zealand', 'cancel_public_events_new_zealand'),
 ('cancel_public_events_new_zealand', 'restrict_public_gathering_new_zealand'),
 ('vaccine_policy_new_zealand', 'cases_per_mil_cat_malaysia'),
 ('vaccine_policy_new_zealand', 'restriction_internal_movement_united_states'),
 ('vaccine_policy_new_zealand', 'income_support_south_africa')]

Singapore, New Zealand had very similar measures when it came to covid control. And this was rightly inferred from the bayesian network model that we built as we got f1 scores close to 1 when we tried to infer travel control measures of 
New Zealand given Singapore.